In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import warnings
warnings.filterwarnings('ignore')

In [2]:
!wget -c https://cdn-datasets.huggingface.co/EsperBERTo/data/oscar.eo.txt

--2021-04-19 17:31:43--  https://cdn-datasets.huggingface.co/EsperBERTo/data/oscar.eo.txt
Resolving cdn-datasets.huggingface.co (cdn-datasets.huggingface.co)... 52.84.161.125, 52.84.161.19, 52.84.161.86, ...
Connecting to cdn-datasets.huggingface.co (cdn-datasets.huggingface.co)|52.84.161.125|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 312733741 (298M) [text/plain]
Saving to: ‘oscar.eo.txt’

oscar.eo.txt        100%[===================>] 298.25M  73.1MB/s    in 4.0s    

2021-04-19 17:31:47 (73.7 MB/s) - ‘oscar.eo.txt’ saved [312733741/312733741]



In [4]:
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-7and2aci
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-7and2aci
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.3MB 7.2MB/s 
     |████████████████████████████████| 901kB 51.9MB/s 
  Created wheel for transformers: filename=transformers-4.6.0.dev0-cp37-none-any.whl size=2112144 sha256=1723a32b8cbf7cc85d1f3a6e8d2bd2779f2a9823c63ed24e071c872f022330ed
  Stored in directory: /tmp/pip-ephem-wheel-cache-nwrj_7q8/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
Successfully built transformers
tokenizers                    0.10.2        
transformers                  4.6.0.dev0    


In [5]:
from pathlib import Path

from tokenizers import ByteLevelBPETokenizer

paths = [str(x) for x in Path(".").glob("**/*.txt")]

tokenizer = ByteLevelBPETokenizer()

tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[ "<s>", "<pad>","</s>","<unk>","<mask>",])

In [6]:
tokenizer.save_model("/content/tokenizer_model")

['/content/tokenizer_model/vocab.json', '/content/tokenizer_model/merges.txt']

In [7]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer("/content/tokenizer_model/vocab.json","/content/tokenizer_model/merges.txt")

In [8]:
tokenizer._tokenizer.post_processor = BertProcessing( ("</s>", tokenizer.token_to_id("</s>")),("<s>", tokenizer.token_to_id("<s>")),)

In [9]:
tokenizer.enable_truncation(max_length=512)

In [10]:
import torch
torch.cuda.is_available()

True

In [11]:
from transformers import RobertaConfig

config = RobertaConfig(vocab_size=52_000,max_position_embeddings=514,num_attention_heads=12,num_hidden_layers=6,type_vocab_size=1)

In [12]:

from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("/content/tokenizer_model", max_len=512)

In [13]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

In [14]:
model.num_parameters()

83504416

In [ ]:
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(tokenizer=tokenizer,file_path="/content/oscar.eo.txt",block_size=128,)

In [22]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

In [23]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(output_dir="/content/output_dir",overwrite_output_dir=True,num_train_epochs=1,
per_gpu_train_batch_size=64,save_steps=10_000,save_total_limit=2,prediction_loss_only=True)

In [24]:
trainer = Trainer(model=model,args=training_args,data_collator=data_collator,train_dataset=dataset)

In [ ]:
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


In [ ]:
trainer.save_model("/content/saved_model")